# POM Nettoyage de Donnees:

## Importatation des Bibliotheques

*   Importatation de Pandas

In [3]:
import pandas as pd

*   Importatation de Pandas

In [8]:
import numpy as np

## Imprtation du fichier .csv depuis un url:

* Installation de Google Api Python 
* Installation de Google Authentification
* Installation de Google Api Client

In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [4]:
from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/Account_Service.json'
creds = None
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1aDOaUAy9WiTh3lt8XaojS7IkFmRZhhMsoYcHn4y38gA'

service = build('sheets','v4', credentials=creds)
# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,range="A1:P5728").execute()
values = result.get('values', [])

## Transformation des resultats en Data Frame

In [5]:
colonnes = ['Nom', 'Nom url', 'Developpeur', 'Categorie(s)', 'Nombre installations', 'Date de mise a jour', 'Taille', 'Version Actuelle', 'Version Android', 'Note', 'Date', 'Heure', 'URL Application', 'Pays', 'Categorie Principale Application','Categorie Secondaire Application']
#dataFrameTmp : Data Frame Temporaire
dataFramTmp = pd.DataFrame(data = values , columns = colonnes) 
dataFramTmp = dataFramTmp.iloc[1: , :]
dataFramTmp.columns
df = dataFramTmp

### Affichage de la DataFrame Brute 

In [6]:
df.head()

,Nom,Nom url,Developpeur,Categorie(s),Nombre installations,Date de mise a jour,Taille,Version Actuelle,Version Android,Note,Date,Heure,URL Application,Pays,Categorie Principale Application,Categorie Secondaire Application
1,Stumble Guys: Multiplayer Royale,kitkagames.fallbuddies,Kitka Games,Action,50000000,"March 28, 2022",120 Mo,0.37,5.1,4.1,2022-05-20,08:50:08,https://apptica.com/charts/stumble-guys-multip...,France,Game,Action
2,Count Masters - Stickman Clash,crowdrun.com,Freeplay Inc,Action,100000000,"April 21, 2022",98 Mo,1.37.2,5,3.7,2022-05-20,08:50:09,https://apptica.com/charts/count-masters-stick...,France,Game,Action
3,Blob Hero,quok.blobHero,Rollic Games,Action,1000000,"May 18, 2022",96 Mo,2000.5.4,6,3.4,2022-05-20,08:50:09,https://apptica.com/charts/blob-hero/9263297/f...,France,Game,Action
4,Hair Challenge,Upperpik.HairChallenge,Rollic Games,Action,100000000,"May 18, 2022",57 Mo,8.5.2004,4.4,4.2,2022-05-20,08:50:09,https://apptica.com/charts/hair-challenge/8028...,France,Game,Action
5,Brawl Stars,supercell.brawlstars,Supercell,Action,100000000,"May 12, 2022",351 Mo,43.248,4.3,4.2,2022-05-20,08:50:09,https://apptica.com/charts/brawl-stars/1159733...,France,Game,Action


# Nettoyage de Donnes

### Remplacement des valeurs des cellules vide par des Nan 

In [9]:
dfemptyfill = df.replace(r'^\s*$', np.nan, regex=True)
df = dfemptyfill

### Isolation des Applications qui contiennent des valeurs manquantes

In [10]:
missing_valuesdf = df[df.isnull().any(1)]
missing_valuesdf.head()
df = missing_valuesdf

Sauvgarde des valeurs manquantes en csv

In [11]:
from google.colab import files
missing_valuesdf.to_csv('data_containes_missing_values.csv', index = False) 
files.download('data_containes_missing_values.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Les applications qui ne contiennent pas des valeurs manquantes

In [12]:
droped_missing_values = df.dropna()
df = droped_missing_values

### Affichage de la data frame qui ne contient aucune valeurs manquante dans ces cellules

In [13]:
df.head()

,Nom,Nom url,Developpeur,Categorie(s),Nombre installations,Date de mise a jour,Taille,Version Actuelle,Version Android,Note,Date,Heure,URL Application,Pays,Categorie Principale Application,Categorie Secondaire Application


### Isolation des Applications qui contiennent "Varies with devices"

In [14]:
dataFrame_VWD = df[(df['Taille'] == 'Varies with device') | (df['Version Actuelle'] == 'Varies with device') | (df['Version Android'] == 'Varies with device')]

Les applications qui contiennent "Varies with devices"

In [15]:
dataFrame_VWD.head()

,Nom,Nom url,Developpeur,Categorie(s),Nombre installations,Date de mise a jour,Taille,Version Actuelle,Version Android,Note,Date,Heure,URL Application,Pays,Categorie Principale Application,Categorie Secondaire Application


###  Isolation des applications qui ne contiennent pas "Varies with devices"

In [16]:
TempdataF_NVWD_Num0 = df 
TempdataF_NVWD_Num1 = TempdataF_NVWD_Num0.drop(TempdataF_NVWD_Num0.index[TempdataF_NVWD_Num0['Taille'] == 'Varies with device'])
TempdataF_NVWD_Num2 = TempdataF_NVWD_Num1.drop(TempdataF_NVWD_Num1.index[TempdataF_NVWD_Num1['Version Actuelle'] == 'Varies with device'])
TempdataF_NVWD_Num3 = TempdataF_NVWD_Num2.drop(TempdataF_NVWD_Num2.index[TempdataF_NVWD_Num2['Version Android'] == 'Varies with device'])
df_NoVWD = TempdataF_NVWD_Num3

In [17]:
df = df_NoVWD

### Nettoyage de la colonne Version Actuelle

+ Enlever les espaces 
+ Remplacer la valeur de certaines cellules par '.' 
+ Enlever les pionts doubles et tribles   
+ Enlever les caracteres speciaux et les lettres alphabetiques
+ Enlever les points au dernier 

* Enlever les espaces

In [18]:
space_removing = df
space_removing['Version Actuelle'] = space_removing['Version Actuelle'].str.strip()
df = space_removing

* Remplacement de - | + _  par un point.

In [19]:
special_cara = '+-_'
for special_caracter in special_cara:
  df['Version Actuelle'] = df['Version Actuelle'].str.replace(special_caracter,".", regex = True)

* Enlever les lettre alphabetiques et les caracteres speciaux

In [20]:
alphabet_and_special_caracters = '!#$%^&*@()abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
for spcial_and_caract in alphabet_and_special_caracters:
  df['Version Actuelle'] = df['Version Actuelle'].str.replace(spcial_and_caract,"", regex = True)

df['Version Actuelle'] = df['Version Actuelle'].str.replace(' ',"", regex = True)

* Enlever les pionts au dernier

In [ ]:
for word in  df['Version Actuelle']:
  if word.endswith('.') == True:
    word = word.rstrip('.')
    space_removing['Version Actuelle']

df['Version Actuelle'] = df['Version Actuelle'].map(lambda x: x.rstrip('.'))

## Typage 

In [ ]:
df['Nom'] = df['Nom'].astype(object)
df['Nom url'] = df['Nom url'].astype(object)
df['Developpeur'] = df['Developpeur'].astype(object)
df['Categorie(s)'] = df['Categorie(s)'].astype(object)
df['Nombre installations'] = df['Nombre installations'].astype(int)
df['Date de mise a jour'] = df['Date de mise a jour'].astype('datetime64[ns]')
df['Taille'] = df['Taille'].astype(object)
df['Version Actuelle'] = df['Version Actuelle'].astype(float)
df['Version Android'] = df['Version Android'].astype(float)
df['Note'] = df['Note'].astype(float)
df['Date'] = df['Date'].astype('datetime64[ns]')
df['Heure'] = df['Heure'].astype('datetime64[ns]')
df['URL Application'] = df['URL Application'].astype(object)
df['Pays'] = df['Pays'].astype(object)
df['Categorie Principale Application'] = df['Categorie Principale Application'].astype(object)
df['Categorie Secondaire Application'] = df['Categorie Secondaire Application'].astype(object)

NameError: ignored

### Creation d'une nouvelle colonne  ( Version Majeur )

* Typage de la colone Version Android:


Le typage de la colonne Version Android en Float nous permet de faire des conditions pour pouvoir faire les categoreries les versions majeurs a la suite.

In [ ]:
df['Version Android'] = df['Version Android'].astype(float)

* Les condition pour regrouper les application en fonction de leur version android:

Le regroupement des applications en 8 versions majeures

In [ ]:
condition_versions_majeures = [ ( df['Version Android'] >= 2.0 ) & ( df['Version Android'] < 3.0 ),
                      ( df['Version Android'] >= 3.0 ) & ( df['Version Android'] < 4.0 ),
                      ( df['Version Android'] >= 4.0 ) & ( df['Version Android'] < 5.0 ),
                      ( df['VersiLa suppression deson Android'] >= 5.0 ) & ( df['Version Android'] < 6.0 ),
                      ( df['Version Android'] >= 6.0 ) & ( df['Version Android'] < 7.0 ),
                      ( df['Version Android'] >= 7.0 ) & ( df['Version Android'] < 8.0 ),
                      ( df['Version Android'] >= 8.0 ) & ( df['Version Android'] < 9.0 ),
                      ( df['Version Android'] >= 9.0 ) & ( df['Version Android'] < 10.0 )]
versions_majeures = [ '2','3','4','5','6','7','8','9']

* Creation de la colonne Version Majeurs 

In [ ]:
df['Version Majeur'] = np.select(condition_versions_majeures,versions_majeures)

* Affichage de nouvelle DataFram contentante la colonne Version Majeure

In [ ]:
df = df.reset_index(drop=True)
df.head()

* Affichage des colonnes de la DataFrame

In [ ]:
df.columns

Index(['Nom', 'Nom url', 'Developpeur', 'Categorie(s)', 'Nombre installations',
       'Date de mise a jour', 'Taille', 'Version Actuelle', 'Version Android',
       'Note', 'Date', 'Heure', 'URL Application', 'Pays',
       'Categorie Principale Application', 'Categorie Secondaire Application',
       'Version Majeur'],
      dtype='object')

On voit que les colonnes ne sont pas en ordre 

* Ordonnancement des colonnes.

In [ ]:
column_names = ['Nom', 'Nom url', 'Developpeur', 'Categorie(s)', 'Nombre installations', 'Date de mise a jour', 'Taille', 'Version Actuelle', 'Version Android', 'Version Majeur',  'Note', 'Date', 'Heure', 'URL Application', 'Pays', 'Categorie Principale Application','Categorie Secondaire Application']
df = df.reindex(columns=column_names)
df = pd.DataFrame(df)

# Resultat Finale

## DataFrame Nettoye 

In [ ]:
df.head()

Sauvgarde de la Dataframe nettoyée

In [ ]:
from google.colab import files
df.to_csv('Cleaned_Data.csv', index = False) 
files.download('Cleaned_Data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>